In [1]:
import numpy as np
import pyvista
import matplotlib.pyplot as plt

from mpi4py import MPI
from dolfinx import mesh, la, fem, plot

import dolfinx.fem.petsc as petsc
from petsc4py import PETSc

from ufl import SpatialCoordinate, TrialFunction, TestFunction, inner, dx, dot, grad, sym, Identity, tr, system, as_vector, sqrt

from dolfinx.io import XDMFFile

# Create Mesh

In [2]:
# Create a rectangular mesh: 100mm x 50mm with 20x10 divisions
domain = mesh.create_rectangle(MPI.COMM_WORLD, [[0, 0], [100, 50]], [20, 10])

In [3]:


pyvista.start_xvfb()
grid = pyvista.UnstructuredGrid(*plot.vtk_mesh(domain))
plotter = pyvista.Plotter(window_size=(120, 60))
renderer = plotter.add_mesh(grid, show_edges=True)

In [4]:
plotter.view_xy()
plotter.camera.zoom(2)
plotter.export_html("./tct.html")

In [5]:
%%html
<iframe src='./tct.html' scrolling="no" width="800px" height="400px"></iframe> <!--  # noqa, -->

In [6]:
# Define function space for displacement
V = fem.functionspace(domain, ("CG", 1, (2,)))


In [7]:
# Locate DOFs on the left edge (x = 0)
def left_edge(x):
    return np.isclose(x[0], 0)

# Locate DOFs on the top edge (y = 50)
def top_edge(x):
    return np.isclose(x[1], 50)

# Locate DOFs on the bottom edge (y = 0) for sinusoidal load
def bottom_edge(x):
    return np.isclose(x[1], 0)

# Get boundary DOFs
left_dofs = fem.locate_dofs_geometrical(V, left_edge)
top_dofs = fem.locate_dofs_geometrical(V, top_edge)
bottom_dofs = fem.locate_dofs_geometrical(V, bottom_edge)


In [8]:
# Fixed zero displacement for left and top edges
zero_displacement = fem.Constant(domain, PETSc.ScalarType(0.0))

# Time-dependent displacement at the bottom
t = 0  # Initialize time
amp = 5.0  # Amplitude in mm
omega = 5000  # Frequency in Hz
sinusoidal_disp = fem.Constant(domain, PETSc.ScalarType(0.0))

def update_sinusoidal_disp(t):
    sinusoidal_disp.value = - amp * np.sin(omega * t)

# Apply boundary conditions
# bc_left = fem.dirichletbc(zero_displacement, left_dofs, V.sub(0))   # Fixed left
bc_top = fem.dirichletbc(zero_displacement, top_dofs, V.sub(1))     # Fixed top
bc_bottom = fem.dirichletbc(sinusoidal_disp, bottom_dofs, V.sub(1)) # Sinusoidal

# bcs = [bc_left, bc_top, bc_bottom]
bcs = [bc_top, bc_bottom]

In [9]:
dt = fem.Constant(domain, 1e-5)

# Define trial and test functions
u = TrialFunction(V)
v = TestFunction(V)

f = fem.Constant(domain, (0.0, 0.0))

def epsilon(u):
    return 0.5 * sym(grad(u))

def sigma(u):
    return lambda_ * tr(epsilon(u)) * Identity(2) + 2 * mu * epsilon(u)

# Define material properties
E = 210e9  # Young's modulus (Pa)
nu = 0.3   # Poisson's ratio
rho = 7850 # Density (kg/m³)

# Define stress-strain relation (Plane stress assumption)
lambda_ = (E * nu) / ((1 + nu) * (1 - 2 * nu))  # First Lame parameter
mu = E / (2 * (1 + nu))  # Shear modulus


# Define weak form
F = inner(sigma(u), epsilon(v)) * dx
F -= dt * inner(f, v) * dx

a, L = system(F)


# # Define trial function for bilinear form
# du = TrialFunction(V)

# # Define the bilinear form (stiffness matrix)
# a = inner(sigma(du), epsilon(v)) * dx

# L = fem.form(inner(fem.Constant(domain, PETSc.ScalarType((0.0, 0.0))), v) * dx)
# # L = inner(f_ext, v) * dx

In [10]:
compiled_a = fem.form(a)
A = petsc.assemble_matrix(compiled_a, bcs=bcs)
A.assemble()

compiled_L = fem.form(L)
b = fem.Function(V)

In [11]:
solver = PETSc.KSP().create(domain.comm)
solver.setOperators(A)
solver.setType(PETSc.KSP.Type.CG)
pc = solver.getPC()
pc.setType(PETSc.PC.Type.HYPRE)
pc.setHYPREType("boomeramg")

In [12]:
T = 0.003  # Total time

# Create functions for displacement, velocity, and acceleration
u_n = fem.Function(V)  # Displacement at time step n
v_n = fem.Function(V)  # Velocity at time step n
a_n = fem.Function(V)  # Acceleration at time step n

gamma = fem.Function(V) # Plastic multiplier

u_h = fem.Function(V)

In [13]:
pyvista.start_xvfb(0.5)  # Start virtual framebuffer for plotting
plotter = pyvista.Plotter()
plotter.open_gif("u_time.gif")

In [14]:
topology, cells, geometry = plot.vtk_mesh(V)
grid = pyvista.UnstructuredGrid(topology, cells, geometry)
grid.point_data["u_h"] = u_h.x.array[1::2]

In [15]:
viridis = plt.cm.get_cmap("viridis", 25)
sargs = dict(
    title_font_size=25,
    label_font_size=20,
    fmt="%.2e",
    color="black",
    position_x=0.1,
    position_y=0.8,
    width=0.8,
    height=0.1,
)

In [16]:
renderer = plotter.add_mesh(
    grid,
    show_edges=True,
    lighting=False,
    cmap=viridis,
    scalar_bar_args=sargs,
    clim=[0, 5],
)

In [17]:
plotter.view_xy()
plotter.camera.zoom(1.3)

In [18]:
u_n.x.array[:] = 0  # ✅ Zero initial displacement
v_n.x.array[:] = 0  # ✅ Zero initial velocity
a_n.x.array[:] = 0  # ✅ Zero initial acceleration


with XDMFFile(MPI.COMM_WORLD, "solution.xdmf", "w") as xdmf:
    xdmf.write_mesh(domain)
    
    # Time-stepping loop
    while t < T:
        # Update boundary condition
        update_sinusoidal_disp(t)
    
        # Assemble RHS
        b.x.array[:] = 0
        petsc.assemble_vector(b.x.petsc_vec, compiled_L)
    
        # Apply boundary condition
        petsc.apply_lifting(b.x.petsc_vec, [compiled_a], [bcs])
        b.x.scatter_reverse(la.InsertMode.add)
        fem.petsc.set_bc(b.x.petsc_vec, bcs)
    
        # Update acceleration, velocity, displacement
        a_n.x.array[:] = b.x.array[:]
        v_n.x.array[:] += dt.value * a_n.x.array[:]  
        u_n.x.array[:] += dt.value * v_n.x.array[:]  
    
        # Apply BCs to displacement after update
        fem.set_bc(u_n.x.array[:], bcs)  # Apply BCs to the PETSc vector
    
        # Assemble force vector (explicit method does NOT use A)
        # b = fem.assemble_vector(fem.form(L))  # Correct RHS assembly
        # fem.set_bc(b, bcs, x0=None)  # Apply BCs to force vector
    
        # Solve linear problem
        # solver.solve(b.x.petsc_vec, u_h.x.petsc_vec)
        # u_h.x.scatter_forward()
    
        # Update un
        # u_n.x.array[:] = u_h.x.array
    
        # Update acceleration (assuming lumped mass M = I)
        # a_n.x.array[:] = b.array[:] 
    
        # # Update velocity using explicit integration
        # v_n.x.array[:] += dt * a_n.x.array[:]
    
        # # Update displacement using explicit integration
        # u_n.x.array[:] += dt * v_n.x.array[:]
    
        # # Apply BCs to displacement after update
        # fem.set_bc(u_n.x, bcs)
    
        xdmf.write_function(u_n, t)
    
        plotter.update_scalars(u_n.x.array[1::2], render=False)
        plotter.write_frame()
    
        # Update time
        t += dt.value


error: XDG_RUNTIME_DIR is invalid or not set in the environment.
MESA: error: ZINK: failed to choose pdev
glx: failed to create drisw screen


In [19]:
plotter.close()

In [20]:
w_grid = pyvista.UnstructuredGrid(*plot.vtk_mesh(domain))
w_plotter = pyvista.Plotter(window_size=(800, 400))
w_grid.point_data["u_h_y"] = u_n.x.array[1::2].real
w_plotter.add_mesh(w_grid, show_edges=True, cmap=viridis, scalar_bar_args=sargs)
w_plotter.view_xy()
w_plotter.export_html("./w.html")

In [21]:
%%html
<iframe src='./w.html' scrolling="no" width="800px" height="400px"></iframe> <!--  # noqa, -->

In [22]:
# ✅ Ensure the mesh is passed correctly
# with XDMFFile(MPI.COMM_WORLD, "solution.xdmf", "w") as xdmf:
#     xdmf.write_mesh(domain)  # ✅ First write the mesh
#     xdmf.write_function(u_n)  # ✅ Then write the function

In [23]:
u_n.x.array

array([-3.43721860e+00, -3.43721860e+00, -1.01390787e+05,  4.34531946e+04,
       -3.43721860e+00, -1.44843982e+05, -1.15875185e+05, -1.88297176e+05,
       -3.43721860e+00, -2.75203566e+05,  1.44843982e+04, -5.79375927e+04,
       -2.02781575e+05, -3.43721860e+00, -1.44843982e+05,  0.00000000e+00,
       -2.75203566e+05,  7.24219909e+04, -7.24219909e+04, -3.43721860e+00,
        7.24219909e+04, -1.30359584e+05,  1.44843982e+04, -1.30359584e+05,
       -1.30359584e+05,  1.44843982e+04, -3.43721860e+00, -7.24219909e+04,
        2.89687964e+04, -1.01390787e+05,  1.44843982e+04,  1.44843982e+04,
        0.00000000e+00, -7.24219909e+04, -3.43721860e+00, -1.44843982e+05,
        0.00000000e+00, -7.24219909e+04,  2.25801468e-12, -2.02781575e+05,
        0.00000000e+00,  0.00000000e+00, -2.17265973e+05, -3.43721860e+00,
       -7.24219909e+04,  0.00000000e+00, -2.02781575e+05,  7.24219909e+04,
       -1.15875185e+05,  1.44843982e+05, -2.02781575e+05,  7.24219909e+04,
       -1.44843982e+05, -

In [24]:
print("Function space shape:", V.dofmap.index_map.size_global)  # Should match number of DOFs in 2D
print("Function value size:", u_n.x.array.shape)  # Should be (num_dofs * 2,)


Function space shape: 231
Function value size: (462,)


In [25]:
print("First 10 values of u_n.x.array[:]:", u_n.x.array[:10])


First 10 values of u_n.x.array[:]: [-3.43721860e+00 -3.43721860e+00 -1.01390787e+05  4.34531946e+04
 -3.43721860e+00 -1.44843982e+05 -1.15875185e+05 -1.88297176e+05
 -3.43721860e+00 -2.75203566e+05]


In [26]:
print("u_x values:", u_n.x.array[::2])  # Every 2nd value is u_x
print("u_y values:", u_n.x.array[1::2])  # Every 2nd value is u_y


u_x values: [-3.43721860e+00 -1.01390787e+05 -3.43721860e+00 -1.15875185e+05
 -3.43721860e+00  1.44843982e+04 -2.02781575e+05 -1.44843982e+05
 -2.75203566e+05 -7.24219909e+04  7.24219909e+04  1.44843982e+04
 -1.30359584e+05 -3.43721860e+00  2.89687964e+04  1.44843982e+04
  0.00000000e+00 -3.43721860e+00  0.00000000e+00  2.25801468e-12
  0.00000000e+00 -2.17265973e+05 -7.24219909e+04 -2.02781575e+05
 -1.15875185e+05 -2.02781575e+05 -1.44843982e+05  0.00000000e+00
  1.44843982e+05  1.44843982e+04 -2.02781575e+05  0.00000000e+00
 -3.43721860e+00  0.00000000e+00  7.24219909e+04  0.00000000e+00
 -7.24219909e+04  0.00000000e+00 -3.43721860e+00 -5.79375927e+04
 -4.05563149e+05 -5.79375927e+04  0.00000000e+00  0.00000000e+00
  4.34531946e+04  7.24219909e+04  0.00000000e+00 -2.02781575e+05
 -5.79375927e+04 -3.43721860e+00 -2.02781575e+05  0.00000000e+00
 -7.24219909e+04  4.34531946e+04  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00 -2.60719167e+05
 -3.43721860e

In [27]:
print(b)

f
